In [1]:
%%python
def gauss_jordan(matrix):
    rows, cols = matrix.rows, matrix.cols
    
    # Forward elimination phase to transform the matrix into row echelon form
    for i in range(rows):
        # Find the maximum element in the current column
        max_row = max(range(i, rows), key=lambda r: abs(matrix[r, i]))
        
        # Swap the current row with the row having the maximum element in current column
        matrix.value[i], matrix.value[max_row] = matrix.value[max_row], matrix.value[i]
        
        # Make the diagonal element 1 and all elements above and below it 0
        for j in range(i + 1, rows):
            factor = matrix[j, i] / matrix[i, i]
            for k in range(i, cols):
                matrix[j, k] -= factor * matrix[i, k]
                
        # Normalize the row
        div = matrix[i, i]
        for k in range(i, cols):
            matrix[i, k] /= div
            
    # Backward elimination phase to transform the matrix into reduced row echelon form
    for i in range(rows - 1, 0, -1):
        for j in range(i):
            factor = matrix[j, i]
            for k in range(cols):
                matrix[j, k] -= factor * matrix[i, k]
                
    return matrix

In [2]:
%%python
import numpy as np
from timeit import timeit

class Matrix:
    def __init__(self, value, rows, cols):
        self.value = value
        self.rows = rows
        self.cols = cols
        
    def __getitem__(self, idxs):
        return self.value[idxs[0]][idxs[1]]
    
    def __setitem__(self, idxs, value):
        self.value[idxs[0]][idxs[1]] = value

def benchmark_gauss_jordan_python(M, N):
    A = Matrix(list(np.random.rand(M, N)), M, N)
    secs = timeit(lambda: gauss_jordan(A), number=2)/2
    gflops = ((2*M*N*(M+N))/secs) / 1e9
    print(gflops, "GFLOP/s")
    return gflops

In [3]:
python_gflops = benchmark_gauss_jordan_python(128, 128).to_float64()

0.009171268860225533 GFLOP/s


In [4]:
#|code-fold: true
#|code-summary: "Import utilities and define `Matrix` (click to show/hide)"

from benchmark import Benchmark
from sys.intrinsics import strided_load
from utils.list import VariadicList
from math import div_ceil, min
from memory import memset_zero
from memory.unsafe import DTypePointer
from random import rand, random_float64
from sys.info import simdwidthof
from runtime.llcl import Runtime
from math import abs

In [ ]:
fn matrix_getitem(self: object, i: Int, j: Int) raises -> Float64:
    return self.value[i][j]

fn matrix_setitem(self: object, i: object, value: object) raises -> object:
    self.value[i] = value
    return None

fn matrix_append(self: object, value: object) raises -> object:
    self.value.append(value)
    return None

fn matrix_init(rows: Int, cols: Int) raises -> object:
    let value = object([])
    return object(
        Attr("value", value), Attr("__getitem__", matrix_getitem), Attr("__setitem__", matrix_setitem), 
        Attr("rows", rows), Attr("cols", cols), Attr("append", matrix_append),
    )

def gauss_jordan(matrix: object) -> object:
    let rows = matrix.rows
    let cols = matrix.cols
    
    for i in range(rows):
        let max_row = i
        for r in range(i, rows):
            if abs(matrix_getitem(matrix, r, i)) > abs(matrix_getitem(matrix, max_row, i)):
                max_row = r
        
        # Swap the current row with the row having the maximum element in current column
        matrix.value[i], matrix.value[max_row] = matrix.value[max_row], matrix.value[i]
        
        # Make the diagonal element 1 and all elements above and below it 0
        for j in range(i + 1, rows):
            let factor = matrix_getitem(matrix, j, i) / matrix_getitem(matrix, i, i)
            for k in range(i, cols):
                matrix.value[j][k] -= factor * matrix_getitem(matrix, i, k)
                
        # Normalize the row
        let div = matrix_getitem(matrix, i, i)
        for k in range(i, cols):
            matrix.value[i][k] /= div
            
    # Backward elimination phase to transform the matrix into reduced row echelon form
    for i in range(rows - 1, 0, -1):
        for j in range(i):
            let factor = matrix_getitem(matrix, j, i)
            for k in range(cols):
                matrix.value[j][k] -= factor * matrix_getitem(matrix, i, k)
                
    return matrix

def benchmark_gauss_jordan_untyped(M: Int, N: Int, python_gflops: Float64):
    A = matrix_init(M, N)
    for i in range(M):
        a_row = object([])
        for j in range(N):
            a_row.append(random_float64(-5, 5))
        A.append(a_row)

    @parameter
    fn test_fn():
        try:
            _ = gauss_jordan(A)
        except:
            pass

    let secs = Float64(Benchmark().run[test_fn]()) / 1_000_000_000
    _ = A
    let gflops = ((2*M*N*(M+N))/secs) / 1e9
    let speedup : Float64 = gflops / python_gflops
    print(gflops, "GFLOP/s, a", speedup.value, "x speedup over Python")


error: The Mojo REPL has crashed and attempted recovery. If the REPL behaves inconsistently, please restart to ensure correct behavior.
expression failed to parse (no further compiler diagnostics)

In [ ]:
#| CHECK: GFLOP/s
benchmark_gauss_jordan_untyped(128, 128, python_gflops)